In [13]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import Draw, rdFingerprintGenerator
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import rdFMCS

In [14]:
data = pd.read_csv("all_chem_df.csv")
data.head()

,image_name,tags,smiles,Col3
0,pics/0,dermatologic,CC(=O)NC1C(O)OC(CO)C(O)C1O,['dermatologic']
1,pics/1,antiinfective,CCC[C@@]1(CCc2ccccc2)CC(O)=C([C@H](CC)c2cccc(N...,['antiinfective']
2,pics/2,antiinfective,CCCCC(C)C(=O)OC1C(C)C(CC)OC2(CC3CC(C/C=C(\C)CC...,['antiinfective']
3,pics/3,antineoplastic,COc1cc2c(c(OC)c1OC)-c1c(cc3c(c1OC)OCO3)C[C@H](...,['antineoplastic']
4,pics/4,antiinfective respiratorysystem,CC(=O)N[C@@H](CS)C(=O)[O-],"['antiinfective', 'respiratorysystem']"


In [15]:
compounds = []
for _, smiles in data["smiles"].items():
    compounds.append(Chem.MolFromSmiles(smiles))

[09:48:57] Conflicting single bond directions around double bond at index 1.
[09:48:57]   BondStereo set to STEREONONE and single bond directions set to NONE.
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] Conflicting single bond directions around double bond at index 1.
[09:48:57]   BondStereo set to STEREONONE and single bond directions set to NONE.
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] WARNING: not removing hydrogen atom without neighbors
[09:48:57] Conflicting single bond directions around double bond at index 1.
[09:48:57]   BondStereo set to STEREONONE and single bond directions set to NONE.
[09

In [16]:
rdkitGen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)
fingerprints = [rdkitGen.GetFingerprint(mol) for mol in compounds]

In [17]:
print(f"Number of compounds converted: {len(fingerprints)}")
print(f"Fingerprint length per compound: {len(fingerprints[0])}")

Number of compounds converted: 8336
Fingerprint length per compound: 1024


In [ ]:
from rdkit.Chem import Draw

# Render a grid of molecules with their original DataFrame index as the legend
sample_size = 20  # adjust if you want to see more/less molecules
valid_indices = [i for i, mol in enumerate(compounds) if mol is not None][:sample_size]
legends = [str(i) for i in valid_indices]
img = Draw.MolsToGridImage(
    [compounds[i] for i in valid_indices],
    legends=legends,
    molsPerRow=5,
    subImgSize=(200, 200),
)
img